In [ ]:
import numpy as np
from mpi4py import MPI
import time

from dedalus import public as de
from dedalus.extras import flow_tools

import logging
logger = logging.getLogger(__name__)

start_init_time = time.time()

# Domain parameters
Lx, Lz = (288. , 288.) #Physical dimensions of the domain in meters

nx, nz = (64, 64) #Number of modes ( degrees of freedom) along each direction in the domain

# Create bases
x_basis = de.Fourier('x', nx, interval=(0, Lx), dealias=3/2)

z_basis = de.Fourier('z', nz, interval=(0, Lz), dealias=3/2)

# Construct domain
#mesh=[n,m] is for running in parallel , divides domain among n processors in one periodic (x or y, in this case) direction andmprocessors in the other
# number of processors must be equal to n*m

domain = de.Domain([x_basis,z_basis], grid_dtype=np.float64, mesh=[4,8])

#In these equations an overline indicates a horizontal mean and primes indicate deviations from the horizontal mean so that f' = f - f_bar

# Set up IVP
problem = de.IVP(domain, variables=['u', 'w', 'b', 'psi', 'psi_z', 'psi_x', 'w_x', 'u_x', 'b_x', 'u_z', 'w_z', 'b_z', 'u_xx', 'w_xx', 'b_xx', 'u_zz', 'w_zz', 'b_zz'])


#Define constant parameters for the equations of motion
problem.parameters[k_e] = 12*np.pi*Lx #total wavenumber
problem.parameters[r] = 
problem.parameters[r_m] = 0.1 #Rayleigh number
problem.parameters[N0_sq] = 1.0e3 #Buoyancy frequency squared


problem.parameters[U_0] = 0.001 #velocity scale

problem.parameters[L_0] = 1/k_e #length scale

#Make domain dimensions into problem parameters
problem.parameters[Lx] = Lx
problem.parameters[Lz] = Lz

#Define the laplacian operator
problem.substitutions['Lap(A_z,A_x)'] = "dz(A_z) + dx(A_x)"


# Mean fields for analysis
problem.substitutions['u_bar'] = "integ('u', 'x', 'z')/(Lx*Ly)"
problem.substitutions['w_bar'] = "integ('w', 'x', 'z')/(Lx*Ly)"
problem.substitutions['b_bar'] = "integ('b', 'x', 'z')/(Lx*Ly)"

#Define the equations of motion
problem.add_equation("dt(u) - Lap(u,w) + dx(psi_z) = -b_x")
problem.add_equation("dt(w) - Lap(w,w) + dz(psi_z) = -b_z")
problem.add_equation("dt(b) - Lap(b,w) = -N0_sq*w")
problem.add_equation("psi_x - dx(psi) = 0")
problem.add_equation("psi_z - dz(psi) = 0")

#Define the boundary conditions
problem.add_bc("left(u) = 0")
problem.add_bc("right(u) = 0")
problem.add_bc("left(w) = 0")
problem.add_bc("right(w) = 0")
problem.add_bc("left(b) = 0")
problem.add_bc("right(b) = 0")
problem.add_bc("left(psi_z) = 0")
problem.add_bc("right(psi_z) = 0", condition="(nx != 0)")
problem.add_bc("integ(psi_z,'z') = 0", condition="(nx == 0)")

# Build solver
solver = problem.build_solver(de.timesteppers.RK443)

#Initial conditions
x = domain.grid(0)
z = domain.grid(1)
u = solver.state['u']
w = solver.state['w']
b = solver.state['b']
psi = solver.state['psi']

#Initialise fields
A = 0.001 #amplitude of initial perturbation
u['g'] = A*np.sin(2*np.pi*x/Lx)*np.sin(2*np.pi*z/Lz)
w['g'] = A*np.sin(2*np.pi*x/Lx)*np.sin(2*np.pi*z/Lz)
b['g'] = A*np.sin(2*np.pi*x/Lx)*np.sin(2*np.pi*z/Lz)
psi['g'] = A*np.sin(2*np.pi*x/Lx)*np.sin(2*np.pi*z/Lz)

#Initialise timestep
dt = 0.01

#Initialise output files
snapshots = solver.evaluator.add_file_handler('snapshots', iter=10, max_writes=50)
snapshots.add_system(solver.state)

#Initialise analysis tasks
analysis = solver.evaluator.add_file_handler('analysis', iter=10, max_writes=50)
analysis.add_task("integ(u,'x')", name='u_int')
analysis.add_task("integ(w,'x')", name='w_int')
analysis.add_task("integ(b,'x')", name='b_int')
analysis.add_task("integ(psi,'x')", name='psi_int')
analysis.add_task("integ(u**2 + w**2,'x')", name='KE')
analysis.add_task("integ(b**2,'x')", name='PE')
analysis.add_task("integ(u**2 + w**2 + b**2,'x')", name='E_tot')
analysis.add_task("integ(u**2 + w**2 + b**2,'x')/(Lx*Lz)", name='E_tot_norm')
analysis.add_task("integ(u**2 + w**2 + b**2,'x')/(Lx*Lz)", name='E_tot_norm')

#Initialise flow properties
flow = flow_tools.GlobalFlowProperty(solver, cadence=10)
flow.add_property("sqrt(u**2 + w**2 + b**2)/sqrt(KE + PE)", name='Re')


